## Model Definition

In [23]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from functools import partial

In [24]:
def activation_function(activation):
    return nn.ModuleDict([
        ['relu', nn.ReLU(inplace=True)],
        ['leaky_relu', nn.LeakyReLU(negative_slope=0.01, inplace=True)],
        ['selu', nn.SELU(inplace=True)],
        ['none', nn.Identity()]
    ])[activation]

def conv_bn(in_channels, out_channels, conv, *args, **kwargs):
    return nn.Sequential(conv(in_channels, out_channels, *args, **kwargs),
                        nn.BatchNorm2d(out_channels))

In [25]:
class Conv2dAuto(nn.Conv2d):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.padding = (self.kernel_size[0] // 2,
                        self.kernel_size[1] // 2)

In [27]:
conv3x3 = partial(Conv2dAuto, kernel_size=3, bias=False)

In [28]:
class BackBone(nn.ModuleList):
    
    def __init__(self, in_channels, out_channels, activation):
        super().__init__()
        
        

In [29]:
class ResidualBlock(nn.Module):
    
    def __init__(self, in_channels, out_channels, activation='relu'):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.activation = activation_function(activation)
        self.blocks = nn.Identity()
        self.shortcut = nn.Identity()
        
    @property
    def is_shortcut(self):
        return self.in_channels != self.out_channels
    
    def forward(self, x):
        residual = self.shortcut(x) if self.is_shortcut else x
        out = self.blocks(x)
        out += residual
        out = self.activation(x)
        return out

In [45]:
class ResNetResidualBlock(ResidualBlock):
    
    def __init__(self, in_channels, out_channels, expansion=1, downsample=1, conv=conv3x3, *args, **kwargs):
        super().__init__(in_channels, out_channels, *args, **kwargs)
        self.expansion, self.downsample, self.conv = expansion, downsample, conv
        self.shortcut = nn.Sequential(nn.Conv2d(self.in_channels, self.expanded_channels, kernel_size=1,
                                               stride=self.downsample, bias=False),
                                     nn.BatchNorm2d(self.expanded_channels)) if self.is_shortcut else None
        
    @property
    def expanded_channels(self):
        return self.out_channels * self.expansion

    @property
    def is_shortcut(self):
        return self.in_channels != self.out_channels

In [34]:
class ResNetBasicBlock(ResNetResidualBlock):
    
    def __init__(self, in_channels, out_channels, *args, **kwargs):
        super().__init__(in_channels, out_channels, *args, **kwargs)
        self.blocks = nn.Sequential(conv_bn(self.in_channels, self.out_channels, conv=self.conv,
                                           bias=False, stride=self.downsample),
                                   self.activation,
                                   conv_bn(self.out_channels, self.expanded_channels, conv=self.conv,
                                          bias=False))

In [46]:
class ResNetBottleNeckBlock(ResNetResidualBlock):
    
    expansion = 4
    
    def __init__(self, in_channels, out_channels, *args, **kwargs):
        super().__init__(in_channels, out_channels, expansion=4, *args, **kwargs)
        nn.Sequential(conv_bn(self.in_channels, self.out_channels, self.conv, kernel_size=1),
                     self.activation,
                     conv_bn(self.out_channels, self.out_channels, self.conv, kernel_size=3, stride=self.downsample),
                     self.activation,
                     conv_bn(self.out_channels, self.expanded_channels, self.conv, kernel_size=1))

In [47]:
dummy = torch.ones((1, 32, 10, 10))
block = ResNetBottleNeckBlock(32, 64)
block(dummy).shape
print(block)

RuntimeError: The size of tensor a (32) must match the size of tensor b (256) at non-singleton dimension 1

In [11]:
class BasicBlock(nn.ModuleList):
    
    def __init__(self, in_channels, out_channels, activation, downsample=1, expansion=1, conv=conv3x3, *args, **kwargs):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.downsample = downsample
        self.activation = activation_function(activation)
        self.expansion = expansion
        self.conv = conv
        self.block = nn.Sequential(
            conv_bn(self.in_channels, self.out_channels, self.conv,
                   stride=self.downsample, bias=False),
            self.activation)
        self.shortcut = nn.Sequential(
            nn.Conv2d(self.in_channels, self.expanded_channels,
                     kernel_size=1, stride = self.downsample),
            nn.BatchNorm2d(self.expanded_channels))
        
    def forward(x):
        residual = self.shortcut(x) if self.is_shortcut else x
        out = self.blocks(x)
        out += residual
        out = self.activation(out)
        return out
    
    @property
    def is_shortcut(self):
        return self.in_channels != slef.out_channels
        
    @property
    def expanded_channels(self):
        return self.out_channels * self.expansion

In [12]:
BasicBlock(32, 64, 'relu')

BasicBlock(
  (blocks): Identity()
  (activation): ReLU(inplace=True)
  (shortcut): Sequential(
    (0): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

# From Scratch

In [3]:
import torch.nn as nn
import torch.functional as F
import torch, torchvision

In [1]:
def conv_bn(in_channels, out_channels, *args, **kwargs):
    return nn.Sequential(nn.Conv2d(in_channels, out_channels, *args, **kwargs),
                        nn.BatchNorm2d(out_channels))

In [5]:
class BasicBlock(nn.ModuleList):
    
    def __init__(in_channels, out_channels):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.shortcut = nn.Sequential(
                        nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False),
                        nn.BatchNorm2d(out_channels)) if self.is_shortcut else nn.Identity()
        self.block = nn.Sequential(conv_bn(self.in_channels, self.out_channels, kernel_size=1, bias=False),
                                  nn.ReLU(),
                                  conv_bn(self.out_channels, self.out_channels, kernel_size=3, bias=False),
                                  nn.ReLU(),
                                  conv_bn(self.out_channels, self.out_channels, kernel_size=1, bias=False))
        
    @property
    def is_shortcut(self):
        return self.in_channels != self.out_channels
    
    def forward(self, x):
        residual = self.shortcut(x)
        out = self.block(x)
        out += residual
        return out

In [ ]:
############################################################
#  Resnet Graph
############################################################

class SamePad2d(nn.Module):
    """Mimics tensorflow's 'SAME' padding.
    """

    def __init__(self, kernel_size, stride):
        super(SamePad2d, self).__init__()
        self.kernel_size = torch.nn.modules.utils._pair(kernel_size)
        self.stride = torch.nn.modules.utils._pair(stride)

    def forward(self, input):
        in_width = input.size()[2]
        in_height = input.size()[3]
        out_width = math.ceil(float(in_width) / float(self.stride[0]))
        out_height = math.ceil(float(in_height) / float(self.stride[1]))
        pad_along_width = ((out_width - 1) * self.stride[0] +
                           self.kernel_size[0] - in_width)
        pad_along_height = ((out_height - 1) * self.stride[1] +
                            self.kernel_size[1] - in_height)
        pad_left = math.floor(pad_along_width / 2)
        pad_top = math.floor(pad_along_height / 2)
        pad_right = pad_along_width - pad_left
        pad_bottom = pad_along_height - pad_top
        return F.pad(input, (pad_left, pad_right, pad_top, pad_bottom), 'constant', 0)

    def __repr__(self):
        return self.__class__.__name__

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, stride=stride)
        self.bn1 = nn.BatchNorm2d(planes, eps=0.001, momentum=0.01)
        self.padding2 = SamePad2d(kernel_size=3, stride=1)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3)
        self.bn2 = nn.BatchNorm2d(planes, eps=0.001, momentum=0.01)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1)
        self.bn3 = nn.BatchNorm2d(planes * 4, eps=0.001, momentum=0.01)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.padding2(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class ResNet(nn.Module):

    def __init__(self, architecture, stage5=False):
        super(ResNet, self).__init__()
        assert architecture in ["resnet50", "resnet101"]
        self.inplanes = 64
        self.layers = [3, 4, {"resnet50": 6, "resnet101": 23}[architecture], 3]
        self.block = Bottleneck
        self.stage5 = stage5

        self.C1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64, eps=0.001, momentum=0.01),
            nn.ReLU(inplace=True),
            SamePad2d(kernel_size=3, stride=2),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.C2 = self.make_layer(self.block, 64, self.layers[0])
        self.C3 = self.make_layer(self.block, 128, self.layers[1], stride=2)
        self.C4 = self.make_layer(self.block, 256, self.layers[2], stride=2)
        if self.stage5:
            self.C5 = self.make_layer(self.block, 512, self.layers[3], stride=2)
        else:
            self.C5 = None

    def forward(self, x):
        x = self.C1(x)
        x = self.C2(x)
        x = self.C3(x)
        x = self.C4(x)
        x = self.C5(x)
        return x


    def stages(self):
        return [self.C1, self.C2, self.C3, self.C4, self.C5]

    def make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes * block.expansion, eps=0.001, momentum=0.01),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)